In [114]:
##################################################
# running.ipynb (PYTHON)
#     Replication of Belo Lin 2012 RFS.
#     by Xinyu 2022.02.03
##################################################

import pandas as pd
import numpy as np
import wrds
import datetime as dt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import matplotlib.pyplot as plt
import os
import re
import importlib
import computation
import output

pd.set_option("display.max_columns", 50)

In [115]:
###################
# Compustat Block #
###################
# note that in the comp.funda and similar tables, there is no sic but just sich, if one wants to access sic directly like from the web query, one can check the comp.names table
conn=wrds.Connection()
comp = conn.raw_sql(f"""
                    select gvkey, datadate, at, ni, pstkl, txditc, pstkrv, seq, pstk, capx, xrd, dltis, csho, cdvc, sale, ppent, invt, sppe, emp, ebit, sich as sic, cik
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate between '07/01/1965' and '12/31/2019'

                    and at >= '0'
                    and sale>='1'

                    """, date_cols=['datadate'])
# These lines below will cause lose of data records due to missing sich

#                     and not sich between '6000' and '6999'
#                     and not sich between '4900' and '4999'
#                    and cik is not null
comp['year']=comp['datadate'].dt.year

# create preferrerd stock
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])
comp['txditc']=comp['txditc'].fillna(0)

# create book equity
comp['be']=comp['seq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

# number of years in Compustat, important to form a qualified sample
comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()

# drop rows with no be or ni
comp = comp.dropna(subset=['be','at', 'ni'])

# fill nan with zeros for mostly 'sic','cik' 
# although there are nan for r&d and cdvc, atm hold out for this operation
comp[['sic','cik']] = comp[['sic','cik']].fillna(0)
comp[['sic','cik']] = comp[['sic','cik']].astype(int)

Loading library list...
Done


In [116]:
# Price and shares outstanding information is extracted from CRSP for calculating market value of equity.
###################
# CRSP Block      #
###################
# sql similar to crspmerge macro
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,b.siccd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '07/01/1965' and '12/31/2019'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date']) 

# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd', 'siccd']]=crsp_m[['permco','permno','shrcd','exchcd', 'siccd']].astype(int)

# Line up date to be end of month
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])

dlret.permno=dlret.permno.astype(int)
#dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)

# retadj factors in the delisting returns
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

In [117]:
### Aggregate Market Cap ###
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()

# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()

# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])

# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)

# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])

# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

# keep December market cap
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
decme=crsp2[crsp2['month']==12]
decme=decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})

### July to June dates
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()

# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)

# lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

# baseline me
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

# merge result back together
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

decme['year']=decme['year']+1
decme=decme[['permno','year','dec_me']]

# Info as of June
crsp3_jun = crsp3[crsp3['month']==6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])
crsp_jun=crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','siccd','retadj','me','wt','cumretx','mebase','lme','dec_me']]
crsp_jun=crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()

In [118]:
#######################
# CCM Block           #
#######################
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm1=pd.merge(comp[['gvkey','datadate','year','be', 'count','capx', 'xrd', 'dltis', 'csho', 'cdvc', 'sale', 'ppent', 'invt', 'sppe', 'at','ni', 'emp', 'ebit', 'sic','cik']],ccm,how='left',on=['gvkey'])
# this construction of jdate is just to facilitate the merge of me of june from next year to compustat data
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

# set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
ccm2=ccm2[['gvkey','permno','year','datadate','be', 'count','capx','jdate', 'xrd', 'dltis', 'csho', 'cdvc', 'sale', 'ppent', 'invt', 'sppe','at', 'ni', 'emp', 'ebit', 'sic','cik']]

# link comp and crsp
ccm_jun=pd.merge(crsp_jun, ccm2, how='inner', on=['permno', 'jdate'])
ccm_jun['beme']=ccm_jun['be']*1000/ccm_jun['dec_me']

# First use historical Compustat SIC Code
# Then if missing use historical CRSP SIC Code
ccm_jun['sic']=np.where(ccm_jun['sic']>0, ccm_jun['sic'], ccm_jun['siccd'])

In [119]:
# Take from constructed annaul table
belo_rep = ccm_jun[['permno', 'invt', 'capx', 'sppe', 'ppent', 'sale', 'at','ni', 'emp', 'be', 'me', 'jdate', 'beme', 'count']]
belo_rep = belo_rep.sort_values(by=['permno','jdate'])
# adjust for the unit
belo_rep['me'] =  belo_rep['me']/1000
# In case of zero inventory, drop them to avoid inf growth rate, hold on for this for further discussion
# belo_rep = belo_rep[belo_rep['invt']!=0]
belo_rep = belo_rep.reset_index(drop=True)

# import cpi from downloaded csv from FRED: https://fred.stlouisfed.org/series/FPCPITOTLZGUSA
cpi = pd.read_csv('../Data/cpi.csv', parse_dates=['DATE'])  
# Note that this 6+12 adjustment is in line with the jdate treatment
# basically for an annual report of 2019, its jdate will be 2020.06, we hope to deflate it with cpi from 2019 
cpi.DATE = cpi.DATE+MonthEnd(6+12)
cpi = cpi.rename(columns={"DATE":'jdate','FPCPITOTLZGUSA':'cpi'})
cpi.cpi = cpi.cpi/100+1

In [120]:
belo_rep

,permno,invt,capx,sppe,ppent,sale,at,ni,emp,be,me,jdate,beme,count
0,10001,0.128,0.551,NaN,8.216,21.460,12.242,0.669,0.077,7.037,5.822125,1987-06-30,1.014415,0
1,10001,0.113,0.513,NaN,8.214,16.621,11.771,0.312,0.076,7.038,6.200000,1988-06-30,1.207618,1
2,10001,0.103,0.240,NaN,7.984,16.978,11.735,0.564,0.062,7.286,7.007000,1989-06-30,1.145192,2
3,10001,0.229,0.444,0.206,14.160,22.910,18.565,1.208,0.080,8.466,10.052250,1990-06-30,0.818149,3
4,10001,0.220,1.173,0.011,14.426,23.227,18.881,1.131,0.084,9.438,11.266500,1991-06-30,0.942575,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240843,93436,953.675,969.885,0.000,2596.011,3198.356,5849.251,-294.040,10.161,969.906,34096.115501,2015-06-30,0.034696,6
240844,93436,1277.838,1634.850,0.000,5194.737,4046.025,8092.460,-888.663,13.058,1130.989,31420.624019,2016-06-30,0.035855,7
240845,93436,2067.454,1440.471,NaN,15036.917,7000.132,22664.076,-674.914,30.025,4761.695,60339.326986,2017-06-30,0.137924,8
240846,93436,2263.537,4081.354,NaN,20491.616,11758.751,28655.372,-1961.400,37.543,4237.312,58478.464281,2018-06-30,0.080626,9


In [121]:
# merge cpi with belo_rep
belo_repm = pd.merge(belo_rep, cpi, on=['jdate'],  how='left')

# remove the first observation of a firm entering the sample
belo_repm = belo_repm[belo_repm['count']>0]

# fill nan with zeros for mostly sppe 
var_list = ['sppe', 'invt', 'emp']
belo_repm[var_list] = belo_repm[var_list].fillna(0)
# 44000+ invt==0 sample
# belo_repm[belo_repm.invt==0].groupby(['jdate']).permno.nunique()

# remove invt==0 obs
belo_repm = belo_repm[belo_repm.invt!=0]
# remove emp==0 obs
belo_repm = belo_repm[belo_repm.emp>=0.01]


# dropna
# alternatively, I can interpolate by group:
# df.groupby('filename').apply(lambda group: group.interpolate(method='index'))
# according to the labor 2013 paper capex missing is droped 

belo_repm = belo_repm.dropna()
# belo_repm = belo_repm.fillna(0)
# this is to make sure non-subsequent year records are not mistakenly connected 
belo_repm = belo_repm[belo_repm.groupby(['permno'])['count'].diff().fillna(1)==1]

In [124]:
# create coresponding variables:
# ig: inventory investment rate in real term
# hn: employment growth
# ik: physical capital investment rate
# sg: real sales growth in real term
# ns: inventory-to-sales ratio
# roa: Return on assets
# it's important to do groupby for the subtracted term


belo_repm['ig'] = (belo_repm.invt/belo_repm.cpi-belo_repm.groupby(['permno']).invt.shift(1))/belo_repm.groupby(['permno']).invt.shift(1)
# employment growth rate
belo_repm['hn'] = (belo_repm.emp-belo_repm.groupby(['permno']).emp.shift(1))/(belo_repm.emp+belo_repm.groupby(['permno']).emp.shift(1))*2

belo_repm['ik'] = (belo_repm['capx'] - belo_repm['sppe'])/belo_repm.groupby(['permno']).ppent.shift(1)

belo_repm['sg'] = (belo_repm.sale/belo_repm.cpi-belo_repm.groupby(['permno']).sale.shift(1))/belo_repm.groupby(['permno']).sale.shift(1)

belo_repm['ns'] = belo_repm.invt/belo_repm.sale

belo_repm['roa'] = belo_repm.ni/belo_repm['at']



belo_repm['k/me'] = belo_repm.ppent/belo_repm.me

belo_repm['lev'] = (belo_repm['at']-belo_repm.be)/(belo_repm['at']-belo_repm.be+belo_repm.me)

belo_repm['bm'] = belo_repm.be/belo_repm.me

belo_repm['size'] = np.log(belo_repm.me) 

 

belo_repm = belo_repm.dropna()
# belo_repm = belo_repm.fillna(0)

belo_repm.replace([np.inf, -np.inf], 0, inplace=True)

# some code to first calculate mean of each permno and then take average
# wsrz_var = ['ik', 'hn', 'sg', 'ns','permno']
# wsrz = belo_repm[wsrz_var].clip(lower=belo_repm[wsrz_var].quantile(0.01), upper=belo_repm[wsrz_var].quantile(0.99), axis=1)
# des = wsrz.groupby('permno')[['ik', 'hn', 'sg', 'ns']].std()
# pd.DataFrame(des.mean()).unstack()

/usr/local/sas/jupyterhub/prod/venvs/20210519/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/sas/jupyterhub/prod/venvs/20210519/lib/python3.9/site-packages/pandas/core/frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


#### Summary statistics

In [127]:

wsrz_var = ['ig','ik', 'hn', 'sg', 'ns','roa']
sum_num = len(wsrz_var)
# I have to make the winsorizing more strict to get comparable results with Belo
wsrz = belo_repm[wsrz_var].clip(lower=belo_repm[wsrz_var].quantile(0.05), upper=belo_repm[wsrz_var].quantile(0.95), axis=1)

# specify percentile to include in the summary stats
summary_table = wsrz.describe(percentiles =  [.1, .9]).transpose()
# summary_table.columns.tolist()
summary_table['count'] = summary_table['count'].astype(int)
summary_table = summary_table[['mean', 'std', '10%', '90%']]
# make a df to record all means from ac1 calculation
test = pd.DataFrame()
for var in wsrz_var:
    test[var]=belo_repm.groupby('permno')[var].apply(pd.Series.autocorr, lag=1).reset_index(name=var)[var]
# attach the result to summary table
summary_table['ac1'] = test.mean()

# rearrange the sequence of column names 
summary_table = summary_table[['mean', 'std','ac1', '10%', '90%']]
# calculate ac1 for each permno
corrmatrix = belo_repm.groupby('permno')[wsrz_var].corr()
# calculate the average of these correlation coef
corr_matrix = pd.DataFrame(corrmatrix.unstack().mean()).unstack().droplevel(0, axis=1) 
# concat horizontally 
result = pd.concat([summary_table, corr_matrix], axis=1)
# drop ik to be inline with belo's table
result = result.drop(['ik'], axis=1).round(2)

# prepare for the multi index
level1_col = result.columns.tolist()
level0_col = ['empty']*3 + ['Percentile'] *2 + ['Correlations']*(sum_num-1)
arrays = [level0_col , level1_col]
tuples = list(zip(*arrays))
multi_col = pd.MultiIndex.from_tuples(tuples)
result.columns=multi_col

/usr/local/sas/jupyterhub/prod/venvs/20210519/lib/python3.9/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)


In [128]:
result

empty             Percentile       Correlations                        
     mean   std   ac1        10%   90%           ig    hn    sg    ns   roa
ig   0.09  0.33 -0.08      -0.27  0.55         1.00  0.41  0.40  0.38  0.15
ik   0.26  0.23  0.16       0.05  0.61         0.27  0.32  0.28  0.06  0.18
hn   0.04  0.17 -0.02      -0.16  0.28         0.41  1.00  0.47  0.10  0.20
sg   0.08  0.21  0.06      -0.16  0.38         0.40  0.47  1.00 -0.05  0.28
ns   0.15  0.11  0.33       0.02  0.31         0.38  0.10 -0.05  1.00 -0.13
roa  0.03  0.08  0.23      -0.09  0.12         0.15  0.20  0.28 -0.13  1.00

In [ ]:
# here is the combination to generate a latex table for the summary statistics
importlib.reload(output)
from output import Tolatex
Tolatex(result, 'summary_stats.tex')

#### One way sort by inventory growth 

In [98]:
ow_sort = belo_repm.copy().reset_index(drop=True)

# set the number of bins required for sorting
sorting_bin = 5

ow_sort['hnport'] = ow_sort.groupby('jdate')['hn'].apply(pd.qcut, sorting_bin, labels = False)+1
# create positivebmeme and nonmissport variable
ow_sort['posbm']=np.where((ow_sort['beme']>0)&(ow_sort['me']>0)&(ow_sort['count']>=1), 1, 0)
ow_sort['nonmissport']=np.where((ow_sort['hnport']!=''), 1, 0)

In [99]:
ow_sort

,permno,invt,capx,sppe,ppent,sale,at,emp,be,me,jdate,beme,count,cpi,hn,ik,sg,ns,k/me,lev,bm,size,hnport,posbm,nonmissport
0,10001,0.103,0.240,0.000,7.984,16.978,11.735,0.062,7.286,7.007000,1989-06-30,1.145192,2,1.040777,-0.202899,0.029218,-0.018542,0.006067,1.139432,0.388355,1.039817,1.946910,1,1,1
1,10001,0.229,0.444,0.206,14.160,22.910,18.565,0.080,8.466,10.052250,1990-06-30,0.818149,3,1.048270,0.253521,0.029810,0.287257,0.009996,1.408640,0.501160,0.842200,2.307796,5,1,1
2,10001,0.220,1.173,0.011,14.426,23.227,18.881,0.084,9.438,11.266500,1991-06-30,0.942575,4,1.053980,0.048780,0.082062,-0.038087,0.009472,1.280433,0.455974,0.837705,2.421834,4,1,1
3,10001,0.248,1.202,0.013,14.813,23.850,19.599,0.087,10.411,12.631250,1992-06-30,0.667907,5,1.042350,0.035088,0.082421,-0.014897,0.010398,1.172726,0.421096,0.824226,2.536174,4,1,1
4,10001,1.233,1.266,0.001,17.120,22.950,22.286,0.106,10.741,17.985000,1993-06-30,0.710384,6,1.030288,0.196891,0.085398,-0.066024,0.053725,0.951904,0.390958,0.597220,2.889538,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147909,93436,953.675,969.885,0.000,2596.011,3198.356,5849.251,10.161,969.906,34096.115501,2015-06-30,0.034696,6,1.016222,0.537079,0.865259,0.563102,0.298177,0.076138,0.125190,0.028446,10.436939,5,1,1
147910,93436,1277.838,1634.850,0.000,5194.737,4046.025,8092.460,13.058,1130.989,31420.624019,2016-06-30,0.035855,7,1.001186,0.249537,0.629755,0.263534,0.315826,0.165329,0.181373,0.035995,10.355220,5,1,1
147911,93436,2067.454,1440.471,0.000,15036.917,7000.132,22664.076,30.025,4761.695,60339.326986,2017-06-30,0.137924,8,1.012616,0.787642,0.277294,0.708571,0.295345,0.249206,0.228809,0.078915,11.007739,5,1,1
147912,93436,2263.537,4081.354,0.000,20491.616,11758.751,28655.372,37.543,4237.312,58478.464281,2018-06-30,0.080626,9,1.021301,0.222531,0.271422,0.644755,0.192498,0.350413,0.294561,0.072459,10.976414,5,1,1


In [91]:
# store portfolio assignment as of June
june_list = ['hn', 'ik', 'size', 'beme', 'k/me', 'lev', 'permno','jdate', 'hnport','posbm','nonmissport']
june=ow_sort[june_list]
june['ffyear']=june['jdate'].dt.year

# merge back with monthly records
crsp3 = crsp3[['date','permno','shrcd','exchcd','retadj','me','wt','cumretx','ffyear','jdate']]
ccm3=pd.merge(crsp3, 
        june[['hn', 'ik', 'size', 'beme', 'k/me', 'lev', 'permno','ffyear', 'hnport','posbm','nonmissport']], how='left', on=['permno','ffyear'])

# keeping only records that meet the criteria
ccm4=ccm3[(ccm3['wt']>0)& (ccm3['posbm']==1) & (ccm3['nonmissport']==1) & 
          ((ccm3['shrcd']==10) | (ccm3['shrcd']==11))]

<ipython-input-91-96a19d2212fe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  june['ffyear']=june['jdate'].dt.year


In [105]:
############################
# calculate one way sorting portfolio returns #
############################


importlib.reload(computation)
from computation import wavg
from computation import ts_reg
from computation import make_owt

# to match belo's sample period
# vwret = vwret[vwret.jdate<'2009-12-31']

# equal-weighted return
ewret=ccm4.groupby(['jdate','hnport'])[['retadj']].mean().reset_index()
ewret[['hnport']] = ewret[['hnport']].astype(int)

# value-weighted return
vwret=ccm4.groupby(['jdate','hnport']).apply(wavg, 'retadj','wt').to_frame().reset_index().rename(columns={0: 'vwret'})
vwret[['hnport']] = vwret[['hnport']].astype(int)

###################
# time series regression with FF #
###################
# smb	double	Small Minus Big (SMB) (smb)
# hml	double	High Minus Low (HML) (hml)
# mktrf	double	Excess Return on the Market (MKTRF) (mktrf)
# rf	double	Risk-Free Interest Rate (One Month Treasury Bill Rate) (RF) (rf)
# umd	double	Momentum (UMD) (umd)

_ff = conn.get_table(library='ff', table='factors_monthly')
_ff=_ff[['date','smb','hml', 'mktrf', 'rf']]
_ff['jdate']=_ff['date']+MonthEnd(0)

## Create panel a part 1 of table 2
pewret = ewret.pivot(index='jdate', columns='hnport', values='retadj')
# pewret.columns.name = None
# pewret = pewret.reset_index()
panel_a1 = make_owt(pewret, _ff, sorting_bin)


## Create panel a part 2 of table 2
pvwret = vwret.pivot(index='jdate', columns='hnport', values='vwret')
panel_a2 = make_owt(pvwret, _ff, sorting_bin)

## Create panel b of table 2
pb_var = ['hn', 'ik', 'size', 'beme', 'k/me', 'lev']
panel_b = ccm4.groupby(['jdate','hnport'])[pb_var].median().reset_index()
panel_b = panel_b.groupby('hnport')[pb_var].mean().T.round(2)
panel_b['L-H'] = panel_b[1] - panel_b[sorting_bin] 
panel_b['MAE'] = ''
panel_b.columns = panel_a1.columns

In [ ]:
importlib.reload(output)
from output import Tolatex
Tolatex(panel_a1, 'panel_a1.tex')
Tolatex(panel_a2, 'panel_a2.tex')
Tolatex(panel_b, 'panel_b.tex')